# 02 - Data Cleaning

Clean the LendingClub dataset for modeling.

**Steps**:

1. Filter to binary outcomes (Fully Paid / Charged Off)
2. Create target variable
3. Handle missing values
4. Remove data leakage columns


In [21]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

# Paths
RAW_DATA = Path("../data/raw/lending_club_raw.parquet")
INTERIM_DATA = Path("../data/interim")
INTERIM_DATA.mkdir(parents=True, exist_ok=True)

In [22]:
# Load raw data
df = pd.read_parquet(RAW_DATA)
print(f"Loaded data shape: {df.shape}")

Loaded data shape: (2260701, 151)


## 1. Filter to Binary Outcomes

Keep only loans with clear outcomes:

- **Fully Paid**: Loan successfully repaid
- **Charged Off**: Loan defaulted (borrower stopped paying)


In [23]:
# Current distribution
print("Original loan status distribution:")
print(df["loan_status"].value_counts())

Original loan status distribution:
loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64


In [24]:
# Filter to binary outcomes
target_statuses = ["Fully Paid", "Charged Off"]
df = df[df["loan_status"].isin(target_statuses)].copy()

print(f"\nAfter filtering: {df.shape[0]:,} rows")
print(df["loan_status"].value_counts())


After filtering: 1,345,310 rows
loan_status
Fully Paid     1076751
Charged Off     268559
Name: count, dtype: int64


## 2. Create Target Variable


In [25]:
# Create binary target: 1 = Default (Charged Off), 0 = Fully Paid
df["default"] = (df["loan_status"] == "Charged Off").astype(int)

print("Target variable distribution:")
print(df["default"].value_counts())
print(f"\nDefault rate: {df['default'].mean():.2%}")

Target variable distribution:
default
0    1076751
1     268559
Name: count, dtype: int64

Default rate: 19.96%


## 3. Handle Missing Values


In [26]:
# Identify columns with missing values
missing_counts = df.isnull().sum()
missing_pct = (missing_counts / len(df) * 100).sort_values(ascending=False)

print("Columns with >50% missing (will drop):")
high_missing = missing_pct[missing_pct > 50]
print(high_missing)

Columns with >50% missing (will drop):
next_pymnt_d                                  100.000000
member_id                                     100.000000
orig_projected_additional_accrued_interest     99.720585
hardship_type                                  99.572292
hardship_reason                                99.572292
hardship_status                                99.572292
deferral_term                                  99.572292
hardship_amount                                99.572292
hardship_start_date                            99.572292
payment_plan_start_date                        99.572292
hardship_length                                99.572292
hardship_dpd                                   99.572292
hardship_loan_status                           99.572292
hardship_end_date                              99.572292
hardship_payoff_balance_amount                 99.572292
hardship_last_payment_amount                   99.572292
sec_app_mths_since_last_major_derog            99

In [27]:
# Drop columns with >50% missing
cols_to_drop = missing_pct[missing_pct > 50].index.tolist()
df = df.drop(columns=cols_to_drop)
print(f"Dropped {len(cols_to_drop)} columns with >50% missing values")
print(f"Remaining columns: {df.shape[1]}")

Dropped 58 columns with >50% missing values
Remaining columns: 94


In [28]:
# Handle specific missing values

# Employment length: fill with '0 years' (assume no employment history)
if "emp_length" in df.columns:
    df["emp_length"] = df["emp_length"].fillna("0 years")
    # Extract numeric value
    df["emp_length_numeric"] = df["emp_length"].str.extract(r"(\d+)").astype(float)
    df["emp_length_numeric"] = df["emp_length_numeric"].fillna(0)
    print(f"emp_length_numeric: {df['emp_length_numeric'].describe()}")

emp_length_numeric: count    1.345310e+06
mean     5.698035e+00
std      3.735888e+00
min      0.000000e+00
25%      2.000000e+00
50%      6.000000e+00
75%      1.000000e+01
max      1.000000e+01
Name: emp_length_numeric, dtype: float64


In [29]:
# Annual income: fill with median
if "annual_inc" in df.columns:
    median_income = df["annual_inc"].median()
    missing_before = df["annual_inc"].isnull().sum()
    df["annual_inc"] = df["annual_inc"].fillna(median_income)
    print(
        f"annual_inc: filled {missing_before} missing with median ${median_income:,.0f}"
    )

annual_inc: filled 0 missing with median $65,000


In [30]:
# Revolving utilization: fill with median
if "revol_util" in df.columns:
    median_util = df["revol_util"].median()
    missing_before = df["revol_util"].isnull().sum()
    df["revol_util"] = df["revol_util"].fillna(median_util)
    print(f"revol_util: filled {missing_before} missing with median {median_util:.1f}%")

revol_util: filled 857 missing with median 52.2%


In [31]:
# Months since features: fill with -1 (indicates "never occurred")
mths_cols = [c for c in df.columns if "mths_since" in c.lower()]
for col in mths_cols:
    missing_before = df[col].isnull().sum()
    if missing_before > 0:
        df[col] = df[col].fillna(-1)
        print(f"{col}: filled {missing_before} missing with -1")

mths_since_recent_bc: filled 60221 missing with -1
mths_since_recent_inq: filled 174071 missing with -1


In [32]:
# DTI: fill with median
if "dti" in df.columns:
    median_dti = df["dti"].median()
    missing_before = df["dti"].isnull().sum()
    df["dti"] = df["dti"].fillna(median_dti)
    print(f"dti: filled {missing_before} missing with median {median_dti:.1f}")

dti: filled 374 missing with median 17.6


## 4. Remove Data Leakage Columns

**Critical**: Remove columns that contain information not available at loan origination.

These include:

- Payment history (when/how much was paid)
- Collection and recovery information
- Current loan status derived fields


In [33]:
# Data leakage columns to remove
leakage_cols = [
    # Payment information (post-origination)
    "issue_d",
    "last_pymnt_d",
    "last_pymnt_amnt",
    "last_credit_pull_d",
    "next_pymnt_d",
    # FICO scores from last credit pull (post-origination)
    "last_fico_range_high",
    "last_fico_range_low",
    # Collection/recovery (post-default)
    "collection_recovery_fee",
    "recoveries",
    # Total payment info (only known after loan ends)
    "total_pymnt",
    "total_pymnt_inv",
    "total_rec_int",
    "total_rec_late_fee",
    "total_rec_prncp",
    # Outstanding principal (changes over time)
    "out_prncp",
    "out_prncp_inv",
    # Funded amounts (could differ from requested)
    "funded_amnt",
    "funded_amnt_inv",
    # Hardship/settlement (post-origination)
    "hardship_flag",
    "debt_settlement_flag",
    "debt_settlement_flag_date",
    "settlement_status",
    "settlement_date",
    "settlement_amount",
    "settlement_percentage",
    "settlement_term",
    # Original loan status (we have 'default' now)
    "loan_status",
    # Other post-origination
    "pymnt_plan",
    "hardship_type",
    "hardship_reason",
    "hardship_status",
    "hardship_amount",
    "hardship_start_date",
    "hardship_end_date",
    "hardship_length",
    "hardship_dpd",
    "hardship_loan_status",
    "hardship_payoff_balance_amount",
    "hardship_last_payment_amount",
    "orig_projected_additional_accrued_interest",
    "payment_plan_start_date",
]

# Remove existing leakage columns
existing_leakage = [c for c in leakage_cols if c in df.columns]
df = df.drop(columns=existing_leakage, errors="ignore")
print(f"Removed {len(existing_leakage)} data leakage columns")
print(f"Remaining columns: {df.shape[1]}")

Removed 21 data leakage columns
Remaining columns: 74


In [34]:
# Also remove ID columns and URLs (not useful for modeling)
id_cols = [
    "id",
    "member_id",
    "url",
    "desc",
    "title",
    "zip_code",
    "addr_state",
    "emp_title",
]
existing_id_cols = [c for c in id_cols if c in df.columns]
df = df.drop(columns=existing_id_cols, errors="ignore")
print(f"Removed {len(existing_id_cols)} ID/text columns")

Removed 6 ID/text columns


## 5. Final Cleaning Check


In [35]:
# Check remaining missing values
remaining_missing = df.isnull().sum()
remaining_missing = remaining_missing[remaining_missing > 0].sort_values(
    ascending=False
)
print(f"Columns still with missing values: {len(remaining_missing)}")
if len(remaining_missing) > 0:
    print(remaining_missing.head(20))

Columns still with missing values: 37
num_tl_120dpd_2m              117401
mo_sin_old_il_acct            105575
pct_tl_nvr_dlq                 67681
avg_cur_bal                    67549
num_rev_accts                  67528
mo_sin_old_rev_tl_op           67528
mo_sin_rcnt_rev_tl_op          67528
num_actv_bc_tl                 67527
num_tl_90g_dpd_24m             67527
num_rev_tl_bal_gt_0            67527
num_op_rev_tl                  67527
num_il_tl                      67527
num_bc_tl                      67527
num_tl_op_past_12m             67527
num_actv_rev_tl                67527
total_il_high_credit_limit     67527
num_accts_ever_120_pd          67527
num_tl_30dpd                   67527
tot_hi_cred_lim                67527
total_rev_hi_lim               67527
dtype: int64


In [36]:
# Fill remaining numeric columns with median
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

print(f"Filled remaining numeric missing values with median")

Filled remaining numeric missing values with median


In [37]:
# Final summary
print("\n" + "=" * 50)
print("CLEANING SUMMARY")
print("=" * 50)
print(f"Final shape: {df.shape}")
print(f"Target variable: 'default'")
print(f"Default rate: {df['default'].mean():.2%}")
print(f"Total missing values: {df.isnull().sum().sum()}")


CLEANING SUMMARY
Final shape: (1345310, 68)
Target variable: 'default'
Default rate: 19.96%
Total missing values: 0


In [38]:
# List remaining columns
print("\nRemaining columns:")
for i, col in enumerate(df.columns):
    dtype = df[col].dtype
    print(f"{i + 1:3}. {col:40} ({dtype})")


Remaining columns:
  1. loan_amnt                                (float64)
  2. term                                     (object)
  3. int_rate                                 (float64)
  4. installment                              (float64)
  5. grade                                    (object)
  6. sub_grade                                (object)
  7. emp_length                               (object)
  8. home_ownership                           (object)
  9. annual_inc                               (float64)
 10. verification_status                      (object)
 11. purpose                                  (object)
 12. dti                                      (float64)
 13. delinq_2yrs                              (float64)
 14. earliest_cr_line                         (object)
 15. fico_range_low                           (float64)
 16. fico_range_high                          (float64)
 17. inq_last_6mths                           (float64)
 18. open_acc                       

## 6. Save Cleaned Data


In [39]:
# Save to interim folder
output_path = INTERIM_DATA / "lending_club_cleaned.parquet"
df.to_parquet(output_path, index=False)

import os

file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"Saved to: {output_path}")
print(f"File size: {file_size_mb:.1f} MB")

Saved to: ../data/interim/lending_club_cleaned.parquet
File size: 86.8 MB


## Next Steps

Proceed to `03_feature_engineering.ipynb` to:

- Create financial ratios (loan-to-income, payment burden)
- Encode categorical variables
- Prepare final feature set
